### Package import

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid

### Read in data
We have data on the birth weights of 189 babies and demographic characteristics and health indicators for the mothers.  The goal of this study was to identify things that might be associated with a low birth weight, defined here as a birth weight of less than 2.5 Kg.  We will ignore the inferential question and ask whether we can predict low birth weight status based on these features.  The specific variables we have are the age of the mother, mother's weight, mother's race (coded as white, black, or other), mother's smoking status, whether or not the mother has hypertension, whether or not there is a presence of uterine irritability, whether or not the mother had premature labors previously, and the number of physician visits during the first trimester (coded as 0, 1, or 2+).

A couple of things to note:

 * **For today, we have a train/test split but not a validation set.**
 * **I have already standardized the features and transposed everything below so that observations are in columns.**

In [22]:
birthwt = pd.read_csv("http://www.evanlray.com/data/mass/birthwt.csv")

lgl_cols = ['smoke', 'ht', 'ui', 'ptd']
birthwt[lgl_cols] = birthwt[lgl_cols].astype('float')
birthwt = pd.get_dummies(birthwt, drop_first = True)
birthwt.head()

,low,age,lwt,smoke,ht,ui,ptd,race_other,race_white,ftv_1,ftv_2+
0,0,19,182,0.0,0.0,1.0,0.0,0,0,0,0
1,0,33,155,0.0,0.0,0.0,0.0,1,0,0,1
2,0,20,105,1.0,0.0,0.0,0.0,0,1,1,0
3,0,21,108,1.0,0.0,1.0,0.0,0,1,0,1
4,0,18,107,1.0,0.0,1.0,0.0,0,1,0,0


In [23]:
y = birthwt['low'].to_numpy()
#print("y = " + str(y))
X = birthwt.drop('low', axis = 1).to_numpy()
#print("X shape = " + str(X.shape))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train_mean = np.mean(X_train, axis = 0)
X_train_std = np.std(X_train, axis = 0)

X_train = X_train - X_train_mean
X_train = X_train / X_train_std

X_test = X_test - X_train_mean
X_test = X_test / X_train_std

X_train = X_train.T
X_test = X_test.T
y_train = y_train.T
y_test = y_test.T

print(X_train.shape)
print(X_test.shape)

(10, 141)
(10, 48)


### Forward propagation
I have defined a function below to do forward propagation.  This is what you saw how to do for logistic regression in Lab 2.

Defining a function for this is kindof excessive for a model with only 1 layer, but we will see it is very useful for more complicated models.

You don't need to do anything to the code below, just run it.

In [0]:
def forward_prop(b, w, X):
  '''
  Forward propagation for a logistic regression model

  Arguments:
   - b: bias parameter
   - w: column vector of weight parameters
   - X: matrix of features of shape (p, m) (features in rows, observations in columns)
  
  Return:
   - A python dictionary with two entries: a and z.  Each is of shape (1, m)
  '''
  z = b + np.dot(w.T, X)
  a = sigmoid(z)

  return(
    {
      'z': z,
      'a': a
    }
  )

Here's an example of calling the function above:

In [25]:
b = np.array([[0.8]])
w = np.array([[0.0, 0.0, 0.7, 1.9, 0.7, 1.3, -0.4, -1.2, -0.4, 0.2]]).T
forward_results = forward_prop(b, w, X_train)
forward_results['a']
# could also look at forward_results['z']

array([[0.91857908, 0.99950865, 0.22084901, 0.06517141, 0.72870874,
        0.4272329 , 0.22084901, 0.4272329 , 0.5434871 , 0.15080971,
        0.99907229, 0.57757765, 0.78252006, 0.15080971, 0.22649177,
        0.5434871 , 0.22084901, 0.06517141, 0.46140153, 0.9999675 ,
        0.5434871 , 0.76158072, 0.06517141, 0.4272329 , 0.06517141,
        0.06517141, 0.2516598 , 0.5434871 , 0.89368444, 0.06517141,
        0.22084901, 0.46140153, 0.85281862, 0.99219602, 0.99856383,
        0.46140153, 0.06517141, 0.46140153, 0.46140153, 0.22084901,
        0.99950865, 0.72317217, 0.93956551, 0.15080971, 0.2516598 ,
        0.96637472, 0.99943573, 0.99283841, 0.85281862, 0.35130422,
        0.89368444, 0.93063469, 0.5434871 , 0.9612609 , 0.15080971,
        0.46140153, 0.99554075, 0.15080971, 0.15080971, 0.46140153,
        0.22649177, 0.22084901, 0.46140153, 0.97866441, 0.99283841,
        0.93956551, 0.46140153, 0.4272329 , 0.4272329 , 0.22649177,
        0.57757765, 0.6668176 , 0.06517141, 0.15

### Backward propagation
Fill in the function below with code to find and return the partial derivatives of the logistic regression cost function with respect to b and w.

In [0]:
def backward_prop(X, y, forward_prop_cache):
  '''
  Backward propagation for a logistic regression model

  Arguments:
   - X: array of features of shape (p, m) (features in rows, observations in columns)
   - y: array of responses of shape (1, m) (observations in columns)
   - forward_prop_cache: dictionary with elements 'z' and 'a' from forward propagation
  
  Return:
   - A python dictionary with two entries: dJdb of shape (1, 1) and dJdw of shape (p, 1).
  '''
  a = forward_prop_cache['a']
  dJdz = a - y # Calculate dJdz
  dJdb = np.sum(dJdz) # Calculate dJdb
  dJdw = np.dot(X, dJdz.T) # Calculate dJdw
  
  return(
    {
      'dJdb': dJdb,
      'dJdw': dJdw
    }
  )

### Initializing estimation
We need starting values for b and w for estimation.  Initialize b and w to arrays of appropriate shape from a normal distribution with standard deviation 0.1.  If you're not sure how to generate values from a normal distribution, check out the numpy tutorial at https://github.com/mhc-stat344ne-s2020/Python_NumPy_foundations/blob/master/Python.ipynb under the "Creating Arrays" heading.  To get a standard deviation of 0.1, the easiest thing to do is to generate values from a normal distribution with standard deviation 1 and then multiply by 0.1.

In [0]:
np.random.seed(98409)
b = np.random.standard_normal((1, 1)) # Replace None to the left with randomly generated initial values for b
w = np.random.standard_normal((X_train.shape[0], 1)) # Replace None to the left with randomly generated initial values for w

### Gradient Descent
Fill in the missing code in the for loop below to run 10000 iterations of gradient descent.

In [0]:
alpha = 0.1
for i in range(10000):
  forward_prop_cache = forward_prop(b, w, X_train)
  gradients = backward_prop(X_train, y_train, forward_prop_cache)
  b = b - alpha * gradients['dJdb'] # Calculate the update to b.  You will need to use gradients['dJdb']
  w = w - alpha * gradients['dJdw'] # Calculate the update to b.  You will need to use gradients['dJdw']

### Test set accuracy
Find predictions for the test set and calculate the test set accuracy.  I got an accuracy of about 73%.

In [29]:
forward_prop_cache = forward_prop(b, w, X_test) # Call the forward_prop function for X_test
a = forward_prop_cache['a'] # Pull out forward_prop_cache['a']
y_hat = (a >= 0.5).astype(int) # Obtain predicted values for y as integer type
np.mean(y_hat == y_test) # Find your test set classification accuracy

0.7291666666666666